### CSCI E-82  Homework 3

### Due by 10/22/18 at 11:59pm EST to the Canvas dropbox

## This is an individual homework so there should be no collaboration for this homework.


### Under each problem, we have a place for you to write the answer, or write runnable code that will produce the answer.  Show your work.
-----------


## Your Name: 

Paul Washburn

## Problem 1 Climate Change (30 points) 

Scientists and politicians are often at odds on the topic of whether global warming is real and debate the various causes. This problem uses "globalWarm3.csv" data.  This is a real data set.

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline

In [6]:
import os
os.listdir('data')

['timeSeries4.csv', 'globalWarm3.csv']

In [7]:
gw = pd.read_csv('data/globalWarm3.csv')
gw.set_index('Year', inplace=True)
gw

,Temp,CO2,Solar,Transmission,IceShelf
Year,,,,,
1980,19,338.57,1366.51,0.929667,7.85
1981,26,339.92,1366.51,0.929767,7.25
1982,4,341.30,1366.16,0.853067,7.45
1983,25,342.71,1366.18,0.897717,7.52
1984,9,344.24,1365.71,0.916492,7.17
1985,4,345.81,1365.57,0.924425,6.93
1986,12,347.11,1365.59,0.926792,7.54
1987,27,348.72,1365.67,0.929092,7.48
1988,31,351.04,1365.97,0.930125,7.49


### Problem 1a 
Plot a scatter plot of the following variables in a lattice: Temp, CO2, Solar, Transmission, and IceShelf. 
    
The variables represent the following:
- Temp = annual surface temperature measured in $1/100^{\circ} C$ over the 1950-1980 mean.
- Solar = annual mean intensity of sunlight piercing the atmosphere 
- CO2 = annual average fraction CO2 in atmosphere (#molecules/#molecules of dry air) 
- IceShelf = sea ice in 1MM square miles hypothesized to reflect heat 
- Transmission = volcanic MLO transmission data where eruptions release greenhouse gases but also decrease the temperature 




In [8]:
from sklearn.preprocessing import MinMaxScaler

_gw = (MinMaxScaler().fit_transform(gw))

pd.plotting.scatter_matrix(gw, figsize=(15, 15), marker='o',
                                 hist_kwds={'bins': 20}, s=60, alpha=.8)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

### Problem 1b 
Compute a multiple linear regression model of log(Temp) against the other variables. Note that since there are limited number of annual measurements, you cannot run all combinations of variables. In fact, you can only do complete pairwise interactions. Be sure to remove the non-significant variables while still maintaining the hierarchy principle in your final model. You do not need to show full diagnostics for the different models that you try, but do show the equations that you tried.

In [17]:
gw

,Temp,CO2,Solar,Transmission,IceShelf
Year,,,,,
1980,19,338.57,1366.51,0.929667,7.85
1981,26,339.92,1366.51,0.929767,7.25
1982,4,341.30,1366.16,0.853067,7.45
1983,25,342.71,1366.18,0.897717,7.52
1984,9,344.24,1365.71,0.916492,7.17
1985,4,345.81,1365.57,0.924425,6.93
1986,12,347.11,1365.59,0.926792,7.54
1987,27,348.72,1365.67,0.929092,7.48
1988,31,351.04,1365.97,0.930125,7.49


### Problem 1c
Run the diagnostics to determine whether your final model is appropriate.

### Problem 1d 

Describe in what way the model diagnostics are appropriate or not.  Be specific.

### Problem 1e

Using your knowledge of statistics, what would you conclude about climate change?

## Problem 2  Matrix model for regression (8 points)

### Problem 2a  
Using the features that you deemed important in Problem 1, construct the matrix forms of the appropriate variables.  Specifically you will need a matrix X that has the features used in your solution and a Y = logTemp.  Print the head of each of these.

### Problem 2b  
Use the matrix calculation for the pseudo-inverse provided in lecture.  

## Problem 2c
How does the answer in Problem 2b compare to that of 1b?

## Problem 3 Time Series Modeling (40 points)

Use the data timeSeries4.csv for this problem. The data are monthly reports of production.

### Problem 3a
Plot the data and perform an exploratory analysis on the raw time series file. Comment on any trends, outliers, seasonality, whether it's stationary, etc.

### Problem 3b
Using your knowledge of ACF, PACF and other diagnostics, walk us through the selection of an appropriate time series model for the data.  We are interested in both the result and your logical journey to reach that model.  That journey should begin with observations from the ACF and PACF pattern.

### Problem 3c
Apply and show the appropriate diagnostics to the model to assert that it is valid.  Include not just a plot but your interpretation of the plot in your justification.

## Problem 4 (15 points)

For a time series data set, a (2,1,1) was derived with the following coefficients:

const   -0.3916      
ar1      0.9172      
ar2     -0.2390      
ma1      0.4012      

The last 5 points are -104.6, -102.1, -103.2, -109.8, -115.7

Compute the next 3 data points by writing the calculation in python.  Note that this will require not only plugging values into the equation, but also taking the d term of the (p,d,q) ARIMA model into account.  We do not need a general form or function--just the required calculations.

## Problem 5 (2 points)
How many hours did this homework take you?  The answer to this question will not affect your grade.

## Last step (5 points)

Save this notebook as LastnameFirstnameHW3.ipynb such as BradyTom.ipynb. Create a pdf of this notebook named similarly. Submit both the python notebook and the pdf version to the Canvas dropbox. We require both versions.